## Let's practice data CLEANING!

In [0]:
from pyspark.sql import Row

# let's make some bad data XD
bad_data = [
    Row(id=1, name="Alice", age=25, score=90.0),
    Row(id=2, name="Bob", age=None, score=85.0),   
    Row(id=3, name="Charlie", age=35, score=None),
    Row(id=4, name="David", age=None, score=None), 
    Row(id=5, name="Eve", age=40, score=-10.0) 
]

df_bad = spark.createDataFrame(bad_data)
display(df_bad)

In [0]:
# option1: if one field is null -> erase the whole row
df_dropped_any = df_bad.na.drop()

# option2: delete the column that has null
df_dropped_score = df_bad.na.drop(subset=['score'])

display(df_dropped_any)
display(df_dropped_score)

In [0]:
# but if i erase all null, i won't have big data left! 
# -> so let's fill with something...

df_filled_zero = df_bad.na.fill(0)
df_filled_custom = df_bad.na.fill({"age": 0, "score": 50.0})

display(df_filled_zero)
display(df_filled_custom)

In [0]:
from pyspark.sql.functions import avg, col, when

# hmm.. but filling with 0 or 50 is not a good idea.
# how about filling with average of scores?

avg_score = df_bad.filter(col("score") > 0).select(avg("score")).first()[0]
print(f"average of scores: {avg_score}")

df_final_cleaned = df_bad.na.fill({"score": avg_score})

df_final_cleaned = df_final_cleaned.withColumn(
    "score",
    when(col("score") < 0, 0).otherwise(col("score"))
)

display(df_final_cleaned)

## Let's join tables!

In [0]:
# here i made some tables for example
users_data = [(1, "Alice"), (2, "Bob"), (3, "Charlie")]
users_df = spark.createDataFrame(users_data, ["user_id", "name"])

orders_data = [(101, 1, "Laptop"), (102, 2, "Mouse"), (103, 4, "Keyboard")]
orders_df = spark.createDataFrame(orders_data, ["order_id", "user_id", "product"])

print("--- users table ---")
users_df.show()
print("--- orders table ---")
orders_df.show()

In [0]:
# inner join
inner_join_df = users_df.join(orders_df, on="user_id", how="inner")

display(inner_join_df) # Charlie disappears

In [0]:
# left join
left_join_df = users_df.join(orders_df, on="user_id", how="left")

# na.fill(), when().otherwise -> No Order
left_join_cleaned = left_join_df.withColumn(
    "product",
    when(col("product").isNull(), "No Order").otherwise(col("product"))
)

# or, this is also good!
from pyspark.sql.functions import coalesce, lit

left_join_cleaned = left_join_df.withColumn(
    "product", 
    coalesce(col("product"), lit("No Order"))
)


display(left_join_cleaned) # Charlie is alive!